In [2]:
import os
import numpy as np
import pandas as pd
from utils import load_sparse_csr

In [3]:
from sklearn.cross_validation import KFold
from sklearn.svm import SVC
from sklearn import metrics
from scipy.sparse import csr_matrix

In [4]:
SEED_VAL = 200;
WORK_DIR = os.getcwd();
data_subset = "_10Percent"
YELP_DATA_CSV_DIR = os.path.join(WORK_DIR, "data", "csv")
YELP_DATA_SPARSE_MATRIX_DIR = os.path.join(WORK_DIR, "data", "sparse_matrix")
YELP_DATA_WORD_2_VEC_MODEL_DIR = os.path.join(WORK_DIR, "data", "word2vec_model")

In [5]:
# Read data
read_filename = os.path.join(YELP_DATA_CSV_DIR, 'business_review_user' + data_subset + ".csv")
df_data = pd.read_csv(read_filename, engine='c', encoding='utf-8')

/Library/anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (5,9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
def mySVC(feature_matrix_train, y_train, feature_matrix_test, y_test):
    clf = SVC()
    clf.fit(feature_matrix_train, y_train)  #.set_params(kernel='linear')
    
    clf2_predictions = clf.predict(feature_matrix_train)
    train_accuracy = str(metrics.accuracy_score(y_train, clf2_predictions))

    clf_predictions = clf.predict(feature_matrix_test)
    test_accuracy = str(metrics.accuracy_score(y_test, clf_predictions))
    
    return [train_accuracy, test_accuracy]

def myKFoldSVM(X, y, n_fold):
    res_svc_train = []
    res_svc_test = []
    res_svc_train_avg = 0.0
    res_svc_test_avg = 0.0
    kf = KFold(X.shape[0], n_folds=n_fold)
    for train_index, test_index in kf:
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        res_svc = mySVC(X_train, y_train, X_test, y_test)
        
        res_svc_train_avg = res_svc_train_avg + float(res_svc[0])
        res_svc_test_avg = res_svc_test_avg + float(res_svc[1])
        
        res_svc_train.append(res_svc[0])
        res_svc_test.append(res_svc[1])
        
    res_svc_train_avg = res_svc_train_avg / n_fold
    res_svc_test_avg = res_svc_test_avg / n_fold
    
    print "SVC training data accuracy: " + str(res_svc_train)
    print "SVC training data average accuracy: " + str(res_svc_train_avg)
    print "SVC test data accuracy:" + str(res_svc_test)
    print "SVC test data average accuracy: " + str(res_svc_test_avg)
    
y = np.array(df_data.review_stars.copy(), dtype='int32')

NameError: name 'feature_matrix_bag_of_words' is not defined

## Bag of words

In [7]:
spare_matrix_file = os.path.join(YELP_DATA_SPARSE_MATRIX_DIR, "bagWords" + data_subset)
feature_matrix_bag_of_words = load_sparse_csr(spare_matrix_file + ".npz")

In [ ]:
%time myKFoldSVM(feature_matrix_bag_of_words, y, 2)


SVC training data accuracy: ['0.607787161854', '0.606431783151']
SVC training data average accuracy: 0.607109472502
SVC test data accuracy:['0.589604443512', '0.591789642655']
SVC test data average accuracy: 0.590697043084
CPU times: user 5h 10min 19s, sys: 1min 15s, total: 5h 11min 35s
Wall time: 5h 24min 17s


In [ ]:
%time mySVC(feature_matrix_bag_of_words.toarray() , y, feature_matrix_bag_of_words, y)
# type(spare_matrix_file)

## Bag of words + Hand crafting features

In [9]:
spare_matrix_file = os.path.join(YELP_DATA_SPARSE_MATRIX_DIR, "bagWords_feat_add" + data_subset)
feature_matrix_bag_of_words_and_hand_craft_features = load_sparse_csr(spare_matrix_file + ".npz")

In [ ]:
%time myKFoldSVM(feature_matrix_bag_o f_words_and_hand_craft_features, y, 5)

## Word embedding

In [9]:
word2vec_feature_matrix_file = os.path.join(YELP_DATA_WORD_2_VEC_MODEL_DIR, "word2vec_feature_matrix" + data_subset+ ".csv")
feature_matrix_word2vec = np.genfromtxt(word2vec_feature_matrix_file, delimiter=',')     

In [10]:
%time myKFoldSVM(feature_matrix_word2vec, y, 5)

SVC training data accuracy: ['0.402808988764', '0.400561797753', '0.410674157303', '0.404494382022', '0.410674157303']
SVC training data average accuracy: 0.405842696629
SVC test data accuracy:['0.41797752809', '0.426966292135', '0.386516853933', '0.411235955056', '0.386516853933']
SVC test data average accuracy: 0.405842696629


# Word embedding + hand craft features

In [13]:
word2vec_feature_matrix_file = os.path.join(YELP_DATA_WORD_2_VEC_MODEL_DIR, "word2vec_add_feature_matrix" + data_subset+ ".csv")
feature_matrix_word2vec_and_hand_craft_features = np.genfromtxt(word2vec_feature_matrix_file, delimiter=',')     

In [14]:
%time myKFoldSVM(feature_matrix_word2vec_and_hand_craft_features, y, 5)

SVC training data accuracy: ['0.47191011236', '0.472471910112', '0.492134831461', '0.47808988764', '0.491573033708']
SVC training data average accuracy: 0.481235955056
SVC test data accuracy:['0.478651685393', '0.501123595506', '0.444943820225', '0.478651685393', '0.460674157303']
SVC test data average accuracy: 0.472808988764


# Hand craft features

In [21]:
feature_matrix_hand_craft_features = feature_matrix_word2vec_and_hand_craft_features[:,100:104]

In [22]:
%time myKFoldSVM(feature_matrix_hand_craft_features, y, 5)

SVC training data accuracy: ['0.519101123596', '0.519101123596', '0.538202247191', '0.526966292135', '0.534831460674']
SVC training data average accuracy: 0.527640449438
SVC test data accuracy:['0.505617977528', '0.496629213483', '0.43595505618', '0.476404494382', '0.433707865169']
SVC test data average accuracy: 0.469662921348
